In [27]:
import pyodbc
import pandas as pd
from datetime import datetime

In [8]:
def conectar(config):
    cnxn = pyodbc.connect('DSN='+ config[5], SERVER=config[0] + ',' + config[1],UID=config[3], PWD=config[4])
    crsr = cnxn.cursor()
    row  = crsr.execute("SELECT * FROM BIGSQL.RIC_DW.RIC_SBX_DETRAN_REQUERIMENTO").fetchone()
    #cnxn.close()
    if row =='':
        print('Conexão falhou')
    else:
        print('Conexão ok')
        
    return  cnxn

In [3]:
f=open("entrada.csv","r")
lines=f.readlines()
argumento = []
valor =[]
for x in lines:
    argumento.append(x.split(";")[0])
    valor.append(x.split(";")[1].split("\n")[0])               
f.close()   
server = valor[0]
serverport = valor[0] + ',' + valor[1]  #endereço vírgula porta
port =valor[1]
database = valor[2]
username = valor[3]
password = valor[4]
dsn = valor[5]

In [9]:
conexao = conectar(valor) 

Conexão ok


In [75]:
cursor = conexao.cursor()
nomeColunas =[]
for row in cursor.columns(table='RIC_SBX_DETRAN_REQUERIMENTO'):
    nomeColunas.append(row.column_name)
conexao.close()

In [180]:
col=open("colunas.txt",'r')
lines=col.readlines()
colu = []
for x in lines:
    colu.append(x.split("'")[1])
col.close() 
colunasUsadas=colu

In [181]:
dfColunas = pd.DataFrame({'NomeColunaDisponível':nomeColunas})
dfColunas['ColunasUteis']= dfColunas.apply(lambda x: x['NomeColunaDisponível'] in colunasUsadas, axis=1)

In [167]:
stringSelect =''
for x in dfColunas.loc[(dfColunas['ColunasUteis']==True)]['NomeColunaDisponível']:
    stringSelect = stringSelect + str(x) +', '
#stringSelect[0:-2] #pega string sem a ultima vírgula

In [166]:
#Teste
#pesquisa = cursor.execute('SELECT '+ stringSelect[0:-2] + ' FROM RIC_DW.RIC_SBX_DETRAN_REQUERIMENTO').fetchone()

In [ ]:
sql = 'SELECT '+ stringSelect[0:-2] + ' FROM RIC_DW.RIC_SBX_DETRAN_REQUERIMENTO'
print(datetime.now()) #10:43
dfBase = pd.read_sql(sql,conexao)

In [168]:
#dfBase.head()


In [164]:
print(datetime.now())

datetime.datetime(2021, 11, 24, 16, 55, 43, 307699)